In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import functools
import collections
import random

#generate random id
import uuid

#ICC in python
import pingouin as pg

#modelling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, cohen_kappa_score
from xgboost import plot_importance
#catboost, for a better support of categorical data
from catboost import CatBoostClassifier, Pool
#to use all except one core
import multiprocessing

#normality test of transitions
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import anderson
from termcolor import colored

#plot
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\camil\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import is_WG_open, ConfMat, list_of_zones, res_intodico, ShapiroTest, AgostinosK2Test
import config_origins as config

C:\Users\camil\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
random.seed(0)

# Download Input Data

In [4]:
#define parameters
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
dico_garden_opening_hour = config.dico_garden_opening_hour
dico_BatchID_endhour = config.dico_BatchID_endhour
path_extracted_data_cleaning = os.path.join(path_extracted_data,'Cleaning')    
path_extracted_data_cleaning_model = os.path.join(path_extracted_data_cleaning,'model')    
#feature for model
li_cont_select = config.li_cont_select
li_bin = config.li_bin
li_cat_select = config.li_cat_select
path__ = os.path.join(path_extracted_data_cleaning,'1secTS')
#create a director if not existing
if not os.path.exists(path__):
    os.makedirs(path__)

In [38]:
df = pd.read_csv(os.path.join(path_extracted_data_cleaning, id_run+'InputCleaning.csv'), parse_dates=['Timestamp', 'date'], 
                 dayfirst=True, sep=';')
df = shuffle(df)
df['VideoAnalyse'] = df['VideoAnalyse'].map(lambda x: int(x))
df_init = df.copy()
print(df.shape)
df.head(3)

(5356, 34)


,Timestamp,TagID,HenID,Zone,PenID,VideoAnalyse,system,date,type,signalstrength,...,duration,previous_duration,algo_type,previousZone==NextZone,next_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist,BatchID,ObservedZone
2675,2020-10-23 08:19:39,59,hen_11,3_Zone,pen5,0,3 - 5,2020-10-23,students,11.0,...,41.0,100.0,cleaning,True,28.0,1.0,1.0,1.0,NaN,5_Zone
3900,2020-10-22 08:42:03,158,hen_9,3_Zone,pen11,1,10 - 12,2020-10-22,students,15.0,...,497.0,1780.0,verification,True,1967.0,1.0,1.0,1.0,ID34,3_Zone
1180,2020-10-17 10:41:13,48,hen_166,5_Zone,pen5,1,3 - 5,2020-10-17,students,8.0,...,11.0,34.0,cleaning,True,24.0,1.0,1.0,1.0,NaN,5_Zone


# Data preprocessing

In [6]:
#scaling numerical data: necessary due to duration high values
display(df[li_cont_select].head(3))
sc = StandardScaler()
li_scale = [x for x in li_cont_select if x not in li_bin]
df[li_scale] = sc.fit_transform(df[li_scale])
display(df[li_cont_select].head(3))

#display categoricalfeatures
df[li_cat_select].head(3)

,signalstrength,signalstzone2,duration,next_duration,previous_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist
4135,13.0,3.0,161.0,771.0,702.0,0.0,1.0,1.0
4849,16.0,7.0,42.0,13.0,10.0,1.0,1.0,1.0
799,16.0,14.0,53.0,23.0,105.0,1.0,1.0,1.0


,signalstrength,signalstzone2,duration,next_duration,previous_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist
4135,0.448270,-1.16435,-0.035920,0.038983,0.008850,0.0,1.0,1.0
4849,1.169561,-0.12415,-0.051403,-0.058764,-0.038112,1.0,1.0,1.0
799,1.169561,1.69620,-0.049971,-0.057475,-0.031665,1.0,1.0,1.0


,Zone,zone2_match,previous1_zone,next1_zone,PenID
4135,5_Zone,3_Zone,4_Zone,4_Zone,pen4
4849,2_Zone,2_Zone,2_Zone,1_Zone,pen12
799,3_Zone,4_Zone,4_Zone,3_Zone,pen8


# Split into training-validation and testing (holdout dataset)

In [7]:
df_testing = df[df['algo_type']=='verification'].copy()
df_testing['VideoAnalyse'].value_counts()

1    821
0     42
Name: VideoAnalyse, dtype: int64

In [8]:
model = CatBoostClassifier()
model.load_model(os.path.join(path_extracted_data_cleaning_model,'FINAL_Catboost'));

# Relevance on TESTING DATA

In [9]:
##########################################################################################################
################################################ predict #################################################
##########################################################################################################
predictions = model.predict(df_testing.filter(li_cont_select+li_cat_select, axis=1))
prediction_proba = model.predict_proba(df_testing.filter(li_cont_select+li_cat_select, axis=1))
df_testing['model_prediction'] = predictions
df_testing['model_prediction_proba0'] = [round(x[0],2) for x in prediction_proba]
df_testing['model_prediction_proba1'] = [round(x[1],2) for x in prediction_proba]
#df_testing.groupby(['Zone','model_prediction'])['date'].count().reset_index()
#add the zone that would be selected with this algorithm output: if==0, then the true zone is the one before
#df_testing['model_prediction_init'] = df_testing['model_prediction'].astype(str)
df_testing['VideoAnalyse'] = df_testing['VideoAnalyse'].astype(int)
df_testing['model_ObservedZone'] = df_testing.apply(lambda x: x['Zone'] if x['model_prediction']==1 else x['previous1_zone'],
                                                    axis=1)
df_testing['tracking_prediction'] = 1
#save
#df_testing.to_csv(os.path.join(path_extracted_data_cleaning, id_run+'_TestingMethods.csv'), 
#                      index=False, sep=';')
display(df_testing.head(3))

,Timestamp,TagID,HenID,Zone,PenID,VideoAnalyse,system,date,type,signalstrength,...,next2zone==Zone,previous2zone==Zone,zone3_match_exist,BatchID,ObservedZone,model_prediction,model_prediction_proba0,model_prediction_proba1,model_ObservedZone,tracking_prediction
3820,2020-10-23 14:25:49,82,hen_147,3_Zone,pen8,1,8 - 9,2020-10-23,students,0.207840,...,1.0,0.0,1.0,ID31,3_Zone,1,0.00,1.00,3_Zone,1
3669,2020-10-15 17:07:35,3,hen_104,2_Zone,pen3,1,3 - 5,2020-10-15,students,-0.753881,...,0.0,1.0,0.0,ID24,2_Zone,1,0.00,1.00,2_Zone,1
3372,2020-10-16 10:35:53,14,hen_25,5_Zone,pen3,1,3 - 5,2020-10-16,students,-0.273020,...,0.0,0.0,1.0,ID8,5_Zone,1,0.01,0.99,5_Zone,1


In [40]:
df_test = df_testing[df_testing['Timestamp']=='23.10.2020  15:15:40']
display(df_init[df_init['Timestamp']=='23.10.2020  15:15:40'][['Timestamp','HenID',
                                                               'VideoAnalyse']+li_cont_select+li_cat_select])
model.predict_proba(df_test.filter(li_cont_select+li_cat_select, axis=1).values)

,Timestamp,HenID,VideoAnalyse,signalstrength,signalstzone2,duration,next_duration,previous_duration,next2zone==Zone,previous2zone==Zone,zone3_match_exist,Zone,zone2_match,previous1_zone,next1_zone,PenID
3803,2020-10-23 15:15:40,hen_12,0,11.0,8.0,222.0,9.0,22.0,1.0,1.0,1.0,3_Zone,5_Zone,5_Zone,5_Zone,pen5


array([[9.99221567e-01, 7.78432829e-04]])

#visualise how correct is the prediction compared to the certainty of the algo
df_testing['is_correct4plot'] = df_testing.apply(lambda x: x['VideoAnalyse']==x['model_prediction'], axis=1)
display(df_testing[['VideoAnalyse', 'model_prediction_init', 'model_prediction_proba0', 
                    'model_prediction_proba1','is_correct4plot']].head(3))
df_testing.groupby(['is_correct4plot','VideoAnalyse'])[['model_prediction', 'model_prediction_proba0', 
                                                    'model_prediction_proba1']].agg(lambda x: np.median(list(x))).reset_index()

#add rule: if max proba is <0.8 then keep correct record. Can be used to comapre the method!!
df_testing['model_prediction'] = df_testing.apply(lambda x: '1' if max(x['model_prediction_proba0'],
                                                            x['model_prediction_proba1'])<0.8 else x['model_prediction_init'], axis=1)
df_testing['is_correct4plot_'] = df_testing.apply(lambda x: x['VideoAnalyse']==x['model_prediction'], axis=1)
display(df_testing['is_correct4plot'].value_counts(), df_testing['is_correct4plot_'].value_counts())

In [10]:
##########################################################################################################
################################# select pairs' combinations of batches ##################################
##########################################################################################################

#number of combination's pair to run for each size
nbr_comb = 10
#minimum nbr of batches to have in a run
min_nbr_batch = 7 #5 is the minimum for transition reliability, so lets have it to be "6"
#compute all possible batch combinations of any size
li_batch = df_testing['BatchID'].unique()

#list of distinct pairs of batch combinations of different size
li_selected_combination_pairs = []
#to have distinct pairs, we cant do more than half of what we have
for nbr_batch in tqdm.tqdm(range(min_nbr_batch, int(len(li_batch)/2))):
    li_sel = []
    while len(li_sel)!=nbr_comb:
        li_pair1 = random.sample(list(li_batch), nbr_batch)
        li_pair2 = random.sample([x for x in li_batch if x not in li_pair1], nbr_batch) 
        li_sel.append([li_pair1,li_pair2])
    li_selected_combination_pairs.extend(li_sel)
#do the one with all batches for final outputs
li_selected_combination_pairs.append([tuple(li_batch),tuple(li_batch)])
print(li_selected_combination_pairs[-3])
print(li_selected_combination_pairs[-2])

100%|████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 2005.54it/s]

[['ID21', 'ID23', 'ID5', 'ID14', 'ID19', 'ID26', 'ID42', 'ID32', 'ID9', 'ID22', 'ID38', 'ID37', 'ID34', 'ID15', 'ID36', 'ID35', 'ID13', 'ID18', 'ID33', 'ID6'], ['ID27', 'ID20', 'ID41', 'ID29', 'ID8', 'ID10', 'ID24', 'ID17', 'ID16', 'ID1', 'ID39', 'ID30', 'ID7', 'ID28', 'ID25', 'ID2', 'ID40', 'ID12', 'ID11', 'ID3']]
[['ID16', 'ID28', 'ID9', 'ID29', 'ID3', 'ID35', 'ID25', 'ID37', 'ID2', 'ID6', 'ID24', 'ID31', 'ID38', 'ID21', 'ID1', 'ID13', 'ID11', 'ID34', 'ID26', 'ID27'], ['ID30', 'ID19', 'ID7', 'ID4', 'ID17', 'ID23', 'ID36', 'ID42', 'ID32', 'ID14', 'ID8', 'ID39', 'ID22', 'ID18', 'ID5', 'ID10', 'ID20', 'ID41', 'ID15', 'ID40']]


In [19]:
df_testing_ = df_testing[df_testing['BatchID'].isin(['ID30'])].copy()
##########################################################################################################
########################################### create duration TS ###########################################
##########################################################################################################
li_df_dur = []
li_col = ['Zone','VideoAnalyse','model_ObservedZone','ObservedZone','PenID','HenID','Timestamp','BatchID']
for BatchID,df_ in df_testing_[li_col].groupby(['BatchID']):
    df_ = df_.sort_values('Timestamp',ascending=True)
    display(df_.head(3))

    #select the list of timestamp we want (i.e. at one second level)
    mi = min(df_['Timestamp'].tolist())
    ma = max(df_['Timestamp'].tolist())
    #extend the daterange if the batch is in dico_BatchID_endhour
    if BatchID in dico_BatchID_endhour.keys():
        ma = dt.datetime(ma.year,ma.month,ma.day,dico_BatchID_endhour[BatchID],0,0)
    #print(mi, ma)
    Daterange = pd.date_range(start = mi+dt.timedelta(seconds=(60-mi.second)), 
                              end = ma-dt.timedelta(seconds=(ma.second+1)), 
                              freq = 'S') 
    print(Daterange[0:2])
    #print(Daterange[-2:])

    ################### 1 sec ts: tracking method & model method & true observation
    ### extend to 1 seconds level
    #add missing seconds (i.e. all seconds that never had a record) and fillnan with last non-nan values by propagating last 
    # observation forward to next valHenIDmi = min(df_['Timestamp'].tolist())
    df_.set_index('Timestamp', inplace=True)
    df__ = df_.reindex(Daterange, method='ffill').reset_index()
    #display(df__.head(5))
    #display(df__.tail(5))
    df__.rename(columns={'index':'Timestamp'}, inplace=True)
    #display(df__.head(3))
    #df__.to_csv(os.path.join(path__,'1secTimeSeries'+str(BatchID)+'.csv'), sep=';', index=False)
    display(df__.head(3))
    sys.exit()
    
    ################### binning method at 60 sec period
    #First take only the wanted timestamps (1 per seconds)
    #due to the 60 sec period bining, we need to exclude some of the first and last seconds in order to take into account 
    #minutes for which we have the full information only
    DaterangeBIN = [Daterange[i] for i in range(len(Daterange)) if i%60==0]
    #add new timestamp to the initial file
    df_date = pd.DataFrame({'New_Timestamp':DaterangeBIN})
    df_date['New_Timestamp'] = df_date['New_Timestamp'].map(lambda x: pd.to_datetime(x))
    df_bin = pd.merge_asof(df__, df_date, left_on=['Timestamp'], right_on=['New_Timestamp'], direction='forward')

    #Then bin the data
    h = 'Zone'
    df_bin = df_bin[[h,'New_Timestamp']].copy()
    df_bin['nbr_sec'] = 1
    df_bin = df_bin.groupby(['New_Timestamp',h])['nbr_sec'].sum().reset_index() #sum to count as we have seconds
    df_bin = df_bin.groupby(['New_Timestamp'])[[h,'nbr_sec']].agg(lambda x: tuple(x)).reset_index()
    df_bin['most_frequent_zone'] = df_bin.apply(lambda x: x[h][x['nbr_sec'].index(max(x['nbr_sec']))], axis=1)
    df_bin['nbr_duration_per_zone'] = df_bin.apply(lambda x: str({x[h][k]:x['nbr_sec'][k] for k in range(len(x[h]))}), axis=1)
    df_bin['nbr_lost_duration_per_zone'] = df_bin['nbr_duration_per_zone'].map(lambda x: str({z:v for z,v in eval(x).items() if \
                                                                       v!=max(eval(x).values())}))
    df_bin['nbr_lost_duration'] = df_bin['nbr_lost_duration_per_zone'].map(lambda x: sum(eval(x).values()))
    df_bin['perc_lost_duration'] = df_bin['nbr_lost_duration'].map(lambda x: x/60*100)
    df_bin['day'] = df_bin['New_Timestamp'].map(lambda x: dt.datetime(x.year,x.month,x.day))
    #df_bin.to_csv(os.path.join(path__ ,'bining'+str(BatchID)+'.csv'), sep=';', index=False)
    #display(df_bin.head(3))    

    #extend to 1sec ts
    df_bin = df_bin[['most_frequent_zone','New_Timestamp']]
    df_bin.set_index('New_Timestamp', inplace=True)
    df_bin1sec = df_bin.reindex(Daterange, method='ffill').reset_index()
    df_bin1sec.rename(columns={'index':'Timestamp','most_frequent_zone':'binning_zone'}, inplace=True)
    #df_bin1sec.to_csv(os.path.join(path__,'1secTimeSeries_bining'+str(BatchID)+'.csv'), sep=';', index=False)
    #display(df_bin1sec.head(3))

    #join with bining
    df_final = pd.merge(df__, df_bin1sec, on=['Timestamp'], how='outer')
    #df_final.to_csv(os.path.join(path__,'1secTimeSeries_ALL_'+str(BatchID)+'.csv'), sep=';', index=False)
    li_df_dur.append(df_final)
#concat all final 1sec into one big
df_final = pd.concat(li_df_dur)
#df_final.to_csv(os.path.join(path_extracted_data_cleaning,'1secTimeSeries_reliability.csv'), sep=';', index=False)
#display(df_final.head(3))

##########################################################################################################
########################################### create duration TS ###########################################
##########################################################################################################
#in case: total nbr of zone, issue: not enough observatoin for kappa test 
#df_transition = df_final.groupby(['BatchID'])[['Zone','model_ObservedZone','ObservedZone',
#                                               'binning_zone']].agg(lambda x: len(list_of_zones(x))-1).reset_index()

li_df_zone = []
for c in ['Zone','model_ObservedZone','ObservedZone','binning_zone']:
    df_transition_ = df_final.groupby(['BatchID'])[c].agg(lambda x: dict(Counter(list_of_zones(x)[1:]))).reset_index()
    for z in df_final[c].unique():
        df_transition_[z] = df_transition_[c].map(lambda x: x.get(z,0))
    df_ = pd.melt(df_transition_, id_vars=['BatchID'], value_vars=df_final[c].unique())
    df_.rename(columns={'value':c}, inplace=True)
    li_df_zone.append(df_)
df_transition = li_df_zone[0]
for df_ in li_df_zone[1:]:
    df_transition = pd.merge(df_, df_transition, on=['BatchID','variable'], how='outer')
print(df_transition.shape)
df_transition.head(3)
#display(df_transition.head(3))
#df_transition.to_csv(os.path.join(path_extracted_data_cleaning,'reliability_transition.csv'), sep=';', index=False)

print(df_testing_.shape, df_final.shape, df_transition.shape)

,Zone,VideoAnalyse,model_ObservedZone,ObservedZone,PenID,HenID,Timestamp,BatchID
3783,2_Zone,1,2_Zone,2_Zone,pen5,hen_12,2020-10-23 14:15:07,ID30
3784,3_Zone,1,3_Zone,3_Zone,pen5,hen_12,2020-10-23 14:16:16,ID30
3785,3_Zone,1,3_Zone,3_Zone,pen5,hen_12,2020-10-23 14:16:37,ID30


DatetimeIndex(['2020-10-23 14:16:00', '2020-10-23 14:16:01'], dtype='datetime64[ns]', freq='S')


,Timestamp,Zone,VideoAnalyse,model_ObservedZone,ObservedZone,PenID,HenID,BatchID
0,2020-10-23 14:16:00,2_Zone,1,2_Zone,2_Zone,pen5,hen_12,ID30
1,2020-10-23 14:16:01,2_Zone,1,2_Zone,2_Zone,pen5,hen_12,ID30
2,2020-10-23 14:16:02,2_Zone,1,2_Zone,2_Zone,pen5,hen_12,ID30


SystemExit: 

C:\Users\camil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
